In [1]:
import numpy as np

In [2]:
import pandas as pd

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
from sklearn.compose import ColumnTransformer

In [6]:
from sklearn.pipeline import Pipeline

In [7]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [8]:
from sklearn.linear_model import LogisticRegression

In [9]:
from sklearn.ensemble import RandomForestClassifier

In [10]:
from xgboost import XGBClassifier

In [11]:
from sklearn.metrics import (
    roc_auc_score, average_precision_score, confusion_matrix, classification_report, precision_score, recall_score, f1_score
)

In [14]:
import warnings
warnings.filterwarnings('ignore')

In [15]:
RANDOM_STATE = 42

In [1]:
TEST_SİZE = 0.2

In [2]:
COST_FP = 1.0

In [ ]:
COST_FN = 6.0

In [20]:
DATA_PATH = 'data/raw/bank-additional-full.csv'

In [21]:
df = pd.read_csv(DATA_PATH, sep = ';')

In [22]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

In [24]:
df.isna().sum().sort_values(ascending=False)

age               0
job               0
marital           0
education         0
default           0
housing           0
loan              0
contact           0
month             0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
emp.var.rate      0
cons.price.idx    0
cons.conf.idx     0
euribor3m         0
nr.employed       0
y                 0
dtype: int64

In [26]:
df.describe().T.head(20)

,count,mean,std,min,25%,50%,75%,max
age,41188.0,40.024060,10.421250,17.000,32.000,38.000,47.000,98.000
duration,41188.0,258.285010,259.279249,0.000,102.000,180.000,319.000,4918.000
campaign,41188.0,2.567593,2.770014,1.000,1.000,2.000,3.000,56.000
pdays,41188.0,962.475454,186.910907,0.000,999.000,999.000,999.000,999.000
previous,41188.0,0.172963,0.494901,0.000,0.000,0.000,0.000,7.000
emp.var.rate,41188.0,0.081886,1.570960,-3.400,-1.800,1.100,1.400,1.400
cons.price.idx,41188.0,93.575664,0.578840,92.201,93.075,93.749,93.994,94.767
cons.conf.idx,41188.0,-40.502600,4.628198,-50.800,-42.700,-41.800,-36.400,-26.900
euribor3m,41188.0,3.621291,1.734447,0.634,1.344,4.857,4.961,5.045
nr.employed,41188.0,5167.035911,72.251528,4963.600,5099.100,5191.000,5228.100,5228.100


In [27]:
missing_ratio = df.isnull().mean().sort_values(ascending=False)

In [28]:
missing_ratio

age               0.0
job               0.0
marital           0.0
education         0.0
default           0.0
housing           0.0
loan              0.0
contact           0.0
month             0.0
day_of_week       0.0
duration          0.0
campaign          0.0
pdays             0.0
previous          0.0
poutcome          0.0
emp.var.rate      0.0
cons.price.idx    0.0
cons.conf.idx     0.0
euribor3m         0.0
nr.employed       0.0
y                 0.0
dtype: float64

In [29]:
df['y'] = (df['y'] == 'yes').astype(int)

In [30]:
df['y'].value_counts(), df['y'].value_counts(normalize = True)

(y
 0    36548
 1     4640
 Name: count, dtype: int64,
 y
 0    0.887346
 1    0.112654
 Name: proportion, dtype: float64)

In [31]:
num_cols = df.select_dtypes(include = 'number').columns.tolist()

In [32]:
cat_cols = [c for c in df.columns if c not in num_cols and c != 'y']

In [34]:
num_cols, cat_cols

(['age',
  'duration',
  'campaign',
  'pdays',
  'previous',
  'emp.var.rate',
  'cons.price.idx',
  'cons.conf.idx',
  'euribor3m',
  'nr.employed',
  'y'],
 ['job',
  'marital',
  'education',
  'default',
  'housing',
  'loan',
  'contact',
  'month',
  'day_of_week',
  'poutcome'])

In [35]:
for c in cat_cols[:6]:
    display(df[c].value_counts().head(10))

job
admin.           10422
blue-collar       9254
technician        6743
services          3969
management        2924
retired           1720
entrepreneur      1456
self-employed     1421
housemaid         1060
unemployed        1014
Name: count, dtype: int64

marital
married     24928
single      11568
divorced     4612
unknown        80
Name: count, dtype: int64

education
university.degree      12168
high.school             9515
basic.9y                6045
professional.course     5243
basic.4y                4176
basic.6y                2292
unknown                 1731
illiterate                18
Name: count, dtype: int64

default
no         32588
unknown     8597
yes            3
Name: count, dtype: int64

housing
yes        21576
no         18622
unknown      990
Name: count, dtype: int64

loan
no         33950
yes         6248
unknown      990
Name: count, dtype: int64

In [36]:
def target_rate_by(col, topn=10):
    t = (
        df.groupby(col)['y']
        .agg(['mean', 'count'])
        .sort_values('count', ascending = False)
        .head(topn)
    )
    return t

In [37]:
display(target_rate_by('job'))

,mean,count
job,,
admin.,0.129726,10422
blue-collar,0.068943,9254
technician,0.108260,6743
services,0.081381,3969
management,0.112175,2924
retired,0.252326,1720
entrepreneur,0.085165,1456
self-employed,0.104856,1421
housemaid,0.100000,1060


In [38]:
display(target_rate_by('marital'))

,mean,count
marital,,
married,0.101573,24928
single,0.140041,11568
divorced,0.103209,4612
unknown,0.150000,80


In [39]:
display(target_rate_by('education'))

,mean,count
education,,
university.degree,0.137245,12168
high.school,0.108355,9515
basic.9y,0.078246,6045
professional.course,0.113485,5243
basic.4y,0.102490,4176
basic.6y,0.082024,2292
unknown,0.145003,1731
illiterate,0.222222,18


In [40]:
X = df.drop(columns = ['y'])

In [41]:
y = df['y'].astype(int)

In [44]:
X_train, X_valid, y_train, y_valid = train_test_split(
    X,y,
    test_size = 0.2,
    stratify = y,
    random_state = RANDOM_STATE
)

In [45]:
X_train.shape, X_valid.shape

((32950, 20), (8238, 20))

In [46]:
num_cols = X_train.select_dtypes(include = 'number').columns

In [47]:
cat_cols = X_train.select_dtypes(exclude = 'number').columns

In [48]:
preprocess = ColumnTransformer(
    transformers = [
        ('num', StandardScaler(), num_cols),
        ('cat', OneHotEncoder(handle_unknown = 'ignore'), cat_cols),
    ]
)

In [49]:
models = {
    'logreg': LogisticRegression(max_iter = 2000, class_weight = 'balanced', random_state = RANDOM_STATE),
    'rf': RandomForestClassifier(
        n_estimators = 400,
        random_state = RANDOM_STATE,
        n_jobs = -1,
        class_weight = 'balanced_subsample'
    ),
    'xgb': XGBClassifier(
        n_estimators = 600,
        max_depth = 5,
        learning_rate = 0.05,
        subsample = 0.9,
        colsample_bytree = 0.9,
        eval_metric = 'logloss',
        random_state = RANDOM_STATE
    )
}

In [50]:
results = []

In [51]:
for name,model in models.items():
    pipe = Pipeline([
        ('prep', preprocess),
        ('model', model)
    ])
    pipe.fit(X_train, y_train)
    proba = pipe.predict_proba(X_valid)[:,1]
    roc = roc_auc_score(y_valid,proba)
    pr = average_precision_score(y_valid,proba)
    results.append({
        'model':name,
        'roc_auc': roc,
        'pr_auc':pr,
        'pipe': pipe
    })

In [53]:
res_df = pd.DataFrame(results).sort_values('roc_auc', ascending = False).reset_index(drop = True)

In [54]:
res_df[['model', 'roc_auc', 'pr_auc']]

,model,roc_auc,pr_auc
0,xgb,0.954407,0.690706
1,rf,0.948976,0.677853
2,logreg,0.943838,0.622248


In [55]:
best_row = res_df.iloc[0]

In [56]:
best_name = best_row['model']

In [57]:
best_pipe = best_row['pipe']

In [58]:
best_name

'xgb'

In [59]:
best_proba = best_pipe.predict_proba(X_valid)[:,1]

In [60]:
roc_auc_score(y_valid, best_proba)

0.954407489740082

In [62]:
average_precision_score(y_valid, best_proba)

0.6907062584804758

In [63]:
def threshold_table(y_true,proba,thresholds = None):
    if thresholds is None:
        thresholds = np.arange(0.05,0.96,0.05)
    rows = []
    for t in thresholds:
        pred = (proba >= t).astype(int)
        rows.append({
            'threshold': float(np.round(t,2)),
            'precision': precision_score(y_true, pred, zero_division = 0),
            'recall': recall_score(y_true, pred, zero_division = 0),
            'f1': f1_score(y_true, pred, zero_division = 0),
            'positive_rate': pred.mean()
        })
    return pd.DataFrame(rows)

In [64]:
tt = threshold_table(y_valid,best_proba)

In [65]:
tt.sort_values('f1', ascending = False).head(10)

,threshold,precision,recall,f1,positive_rate
6,0.35,0.609862,0.759698,0.676583,0.140325
7,0.40,0.633937,0.716595,0.672736,0.127337
4,0.25,0.548704,0.843750,0.664968,0.173222
5,0.30,0.568129,0.795259,0.662775,0.157684
3,0.20,0.521628,0.883621,0.656000,0.190823
8,0.45,0.657549,0.647629,0.652552,0.110949
2,0.15,0.487121,0.917026,0.636262,0.212066
9,0.50,0.680503,0.582974,0.627974,0.096504
1,0.10,0.447154,0.948276,0.607735,0.238893
10,0.55,0.713224,0.517241,0.599625,0.081695


In [66]:
def expected_cost(y_true,proba,threshold,cost_fp = 1.0, cost_fn = 5.0):
    pred = (proba >= threshold).astype(int)
    tn,fp,fn,tp = confusion_matrix(y_true,pred).ravel()
    return fp * cost_fp + fn * cost_fn

In [67]:
cost_fp = 1.0

In [68]:
cost_fn = 6.0

In [69]:
tt_cost = tt.copy()

In [70]:
tt_cost['expected_cost'] = tt_cost['threshold'].apply(
    lambda t: expected_cost(y_valid,best_proba,t,cost_fp = cost_fp, cost_fn = cost_fn)
)

In [71]:
tt_cost.sort_values('expected_cost').head(10)

,threshold,precision,recall,f1,positive_rate,expected_cost
2,0.15,0.487121,0.917026,0.636262,0.212066,1358.0
1,0.10,0.447154,0.948276,0.607735,0.238893,1376.0
3,0.20,0.521628,0.883621,0.656000,0.190823,1400.0
4,0.25,0.548704,0.843750,0.664968,0.173222,1514.0
0,0.05,0.388316,0.974138,0.555283,0.282593,1568.0
5,0.30,0.568129,0.795259,0.662775,0.157684,1701.0
6,0.35,0.609862,0.759698,0.676583,0.140325,1789.0
7,0.40,0.633937,0.716595,0.672736,0.127337,1962.0
8,0.45,0.657549,0.647629,0.652552,0.110949,2275.0
9,0.50,0.680503,0.582974,0.627974,0.096504,2576.0


In [73]:
best_t_f1 = tt.sort_values('f1', ascending = False).iloc[0]['threshold']

In [74]:
best_t_cost = tt_cost.sort_values('expected_cost').iloc[0]['threshold']

In [75]:
best_t_f1, best_t_cost

(np.float64(0.35), np.float64(0.15))

In [76]:
def eval_at_threshold(y_true,proba,threshold):
    pred = (proba >= threshold).astype(int)
    tn,fp,fn,tp = confusion_matrix(y_true,pred).ravel()
    return {
        'threshold': threshold,
        'precision': precision_score(y_true, pred, zero_division = 0),
        'recall': recall_score(y_true,pred,zero_division = 0),
        'f1': f1_score(y_true, pred, zero_division = 0),
        'tn':tn, 'fp':fp, 'fn':fn, 'tp':tp,
        'positive_rate': pred.mean()
    }

In [77]:
eval_f1 = eval_at_threshold(y_valid,best_proba,best_t_f1)

In [78]:
eval_cost = eval_at_threshold(y_valid,best_proba,best_t_cost)

In [79]:
pd.DataFrame([eval_f1, eval_cost])

,threshold,precision,recall,f1,tn,fp,fn,tp,positive_rate
0,0.35,0.609862,0.759698,0.676583,6859,451,223,705,0.140325
1,0.15,0.487121,0.917026,0.636262,6414,896,77,851,0.212066


In [80]:
FINAL_THRESHOLD = best_t_cost

In [81]:
final_pred = (best_proba >= FINAL_THRESHOLD).astype(int)

In [83]:
FINAL_THRESHOLD, confusion_matrix(y_valid,final_pred)

(np.float64(0.15),
 array([[6414,  896],
        [  77,  851]]))

In [84]:
classification_report(y_valid,final_pred,digits = 4)

'              precision    recall  f1-score   support\n\n           0     0.9881    0.8774    0.9295      7310\n           1     0.4871    0.9170    0.6363       928\n\n    accuracy                         0.8819      8238\n   macro avg     0.7376    0.8972    0.7829      8238\nweighted avg     0.9317    0.8819    0.8965      8238\n'

## Business Insight — Bank Marketing Term Deposit Targeting

### Objective
The bank runs phone-based campaigns to sell term deposits.  
Our goal is to predict which customers are likely to subscribe (“yes”) so outreach efforts focus on high-potential leads.

### Data notes
- Dataset size: 41,188 rows
- Target is imbalanced: ~11.3% positive (“yes”)
- Missing values: none

> Operational note: The feature `duration` (call duration) may be known only after the call.  
> If the business goal is “who to call before calling”, the model should be re-trained without `duration` to avoid leakage.

### Model comparison
We trained and compared:
- Logistic Regression
- Random Forest
- XGBoost

Best model on validation: **XGBoost**
- ROC-AUC: **0.9544**
- PR-AUC: **0.6907**

### Threshold selection (why it matters)
Using the default threshold (0.50) is not optimal under class imbalance.  
We evaluated thresholds and selected an operating point based on business priorities.

**Option A — Best F1 (balanced precision/recall)**
- Threshold: **0.35**
- Precision: **0.6099**
- Recall: **0.7597**
- F1: **0.6766**
- Predicted positive rate (outreach rate): **14.0%**

**Option B — Cost-aware threshold (prioritizing missed opportunities)**
Assumption: Missing a true subscriber is more costly than making an unnecessary call  
(cost_fn = 6, cost_fp = 1).
- Threshold: **0.15**
- Precision: **0.4871**
- Recall: **0.9170**
- F1: **0.6363**
- Outreach rate: **21.2%**
- Confusion Matrix (t=0.15): TN=6414, FP=896, FN=77, TP=851

### Recommendation
- If the call center capacity is limited and you want a balanced tradeoff: use **t = 0.35** (higher precision, fewer calls).
- If the business prefers capturing as many potential subscribers as possible (and can tolerate more calls): use **t = 0.15** (higher recall).